In [ ]:
import pandas as pd
from db.settings import PgConnector


query = """
SELECT "CODIGO_CURSO", "NOME_CURSO", "SIGLA_IES", "MUNICIPIO_CAMPUS", count(*) as "INSCRITOS"
	FROM regular_2022
	where "EDICAO" = 1 and "SIGLA_IES" = 'IFPB'
	group by "CODIGO_CURSO", "NOME_CURSO", "SIGLA_IES", "MUNICIPIO_CAMPUS"
order by "INSCRITOS" desc
"""
df = pd.read_sql_query(query, PgConnector().engine)

# SELECT "CODIGO_CURSO", "NOME_CURSO", "TIPO_MOD_CONCORRENCIA", "QT_VAGAS_CONCORRENCIA",
# 	"EDICAO", count(*) as "INSCRITOS"
# 	FROM regular_2022
# 	where "EDICAO" = 1
# 	group by "CODIGO_CURSO", "NOME_CURSO", "TIPO_MOD_CONCORRENCIA", "QT_VAGAS_CONCORRENCIA", "EDICAO"
# order by "INSCRITOS" desc


In [ ]:
import pandas as pd
from db.settings import PgConnector


query = """
select "NOME_CURSO", "EDICAO", count(*) as "INSCRITOS"
	from regular_2022
	group by "NOME_CURSO", "EDICAO"
order by "INSCRITOS" desc
"""
df = pd.read_sql_query(query, PgConnector().engine)
display(df.head(15))

QUANTIDADE INSCRICOES POR CURSO_NOME POR ANO


In [ ]:
import pandas as pd
from db.settings import PgConnector


nome_tabela = "inscricoes_por_curso"

anos_tabelas = [
    "chamada_regular_2017_1",
    "chamada_regular_2018_1",
    "chamada_regular_2019_1",
    "chamada_regular_2020_1",
    "chamada_regular_2021_1",
    "chamada_regular_2022_1",
]
df = pd.DataFrame()
# acredito que faz mais sentido analisar o primeiro semestre nesse caso
# o segundo semestre sempre tem menos inscrições
for ano in anos_tabelas:
    query = """
        SELECT "NOME_CURSO", "ANO", count(*) as "QTD_INSCRICOES"
        from {table}
        group by "NOME_CURSO", "ANO"
        """.format(
        table=ano
    )

    df_ano = pd.read_sql_query(
        query,
        PgConnector().engine,
    )
    df = pd.concat([df, df_ano])

display(df.head(15))

QUANTIDADE DE INSCRITOS POR ANO


In [ ]:
import pandas as pd
from db.settings import PgConnector


nome_tabela = "inscritos_por_ano"

tabelas = [
    "regular_2017",
    "regular_2018",
    "regular_2019",
    "regular_2020",
    "regular_2021",
    "regular_2022",
]
df = pd.DataFrame()
for tabela in tabelas:
    query = """
        select "ANO", count(*) as "INSCRITOS" 
	    from (
		select distinct on 
			("INSCRITO", "INSCRICAO_ENEM", "DATA_NASCIMENTO") "ANO"
		from {table})
	    group by "ANO"
        """.format(
        table=tabela
    )

    df_tabela = pd.read_sql_query(
        query,
        PgConnector().engine,
    )
    df = pd.concat([df, df_tabela])

### QUANTIDADE DE INSCRITOS POR CURSO ESPECÍFICO POR ANO

considerando inscrições para primeira opção

In [ ]:
import pandas as pd
from db.settings import PgConnector


nome_tabela = "inscritos_curso_especifico_etapa1"
tabelas = [
    "chamada_regular_2017_1",
    "chamada_regular_2018_1",
    "chamada_regular_2019_1",
    "chamada_regular_2020_1",
    "chamada_regular_2021_1",
    "chamada_regular_2022_1",
]
df = pd.DataFrame()
for tabela in tabelas:
    query = """
    SELECT "ANO", "NOME_CURSO", "NOME_IES", "NOME_CAMPUS",
        count(*) as "INSCRITOS"
    FROM {tabela}
    WHERE "OPCAO" = 1
    group by "ANO", "NOME_CURSO", "CODIGO_CURSO",
        "CODIGO_IES", "NOME_IES",
        "CODIGO_CAMPUS", "NOME_CAMPUS"
    """.format(
        tabela=tabela
    )

    df_tabela = pd.read_sql_query(
        query,
        PgConnector().engine,
    )
    df = pd.concat([df, df_tabela])
# query anterior, invalida. não combina NOME_IES e NOME_CAMPUS
# select "ANO", "CODIGO_CURSO", "NOME_CURSO",  count(*) as "INSCRITOS"
#     from (
#     select distinct on
#         ("INSCRITO", "INSCRICAO_ENEM", "DATA_NASCIMENTO", "CODIGO_CURSO")
#         "ANO", "CODIGO_CURSO", "NOME_CURSO"
#         from {tabela}
#         )
#     group by "ANO", "CODIGO_CURSO", "NOME_CURSO"

In [ ]:
display(df.sort_values(by="INSCRITOS", ascending=False).head(10))

Salva dados processados em formato .csv e no banco 

In [ ]:
import pandas as pd
from db.settings import PgConnector
import pathlib

PATH = pathlib.Path(".")
csv_filename = f"{nome_tabela}.csv"
view_path = PATH / "transformado" / "views" / csv_filename
try:
    df.to_sql(con=PgConnector().engine, name=nome_tabela, if_exists="replace", index=False)
    df.to_csv(
        view_path,
        index=False,
        sep="|",
        encoding="utf-8",
    )
    
except Exception as e:
    print(e)

visualizando dados com matplotlib

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from db.settings import PgConnector


df = pd.read_sql_table("inscritos_por_ano", PgConnector().engine)
# display(df.head(15))

df.plot(x='ANO', y='INSCRITOS', kind='bar')
plt.xlabel('ANO')
plt.ylabel('INSCRITOS')
plt.title('Numero de inscritos por ano')
plt.show()

### Quantidade de inscritos por região

In [1]:
import pandas as pd
from db.settings import PgConnector


nome_tabela = "inscritos_regiao_quantidade"
tabelas = [
    "regular_2017_1_com_regiao",
    "regular_2018_1_com_regiao",
    "regular_2019_1_com_regiao",
    "regular_2020_1_com_regiao",
    "regular_2021_1_com_regiao",
    "regular_2022_1_com_regiao",
]
df = pd.DataFrame()
connector = PgConnector()


In [ ]:
for tabela in tabelas:
    query = """
    select "ANO", "REGIAO_IES", count(*) as inscricoes 
	from {tabela}
	-- where "OPCAO" = 1
	group by "ANO", "REGIAO_IES"
    """.format(
        tabela=tabela
    )

    df_tabela = pd.read_sql_query(
        query,
        connector.engine,
    )
    df = pd.concat([df, df_tabela])

In [ ]:
import pathlib


PATH = pathlib.Path(".")
csv_filename = f"{nome_tabela}.csv"
view_path = PATH / "transformado" / "views" / csv_filename
print(csv_filename, view_path)

df.to_sql(con=connector.engine, name=nome_tabela, if_exists="fail", index=False)

# df.to_csv(
#     view_path,
#     index=False,
#     sep="|",
#     encoding="utf-8",
# )

In [ ]:
df = pd.read_sql_table("inscritos_por_ano", PgConnector().engine)
